## Part 1 - Extraction and Frequencies

In [5]:
import nltk
from nltk.collocations import *
from datetime import datetime

In [6]:
extraction_start = datetime.now()

file = open("Corpus/maltese-1.txt")
corpus = file.read()

tokens = nltk.word_tokenize(corpus)
extraction_end = datetime.now()

extraction_time = dict()
extraction_time['extraction_time'] = extraction_end - extraction_start
print('extraction_time(HH::MM:SS:ms) - {}\n\n'.format(extraction_time['extraction_time']))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\franc/nltk_data'
    - 'C:\\Users\\franc\\anaconda3\\nltk_data'
    - 'C:\\Users\\franc\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\franc\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\franc\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
